Intention of this notebook is to see how the performance (as measured by *AUC* or *Pearson Correlation Coefficient*) of the models varies if the crossvalidation happens on a per protein basis.

This was done after the main per-protein cross-validation (*leave-one-protein-out cross-validation*) was performed, since the prediction werent saved, only the *MAE* and *RMSE* metrics, which made these calculation impossible retrospectively.

## Notebook Setup


In [0]:
#Imports:
import os
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import RepeatedKFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from sklearn import metrics
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
#import seaborn as sns

import time
import datetime

In [0]:
#Connect to google drive:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
print("Check correctnes of locations: ")

print("Folders: ")
data_target_location = "gdrive/My Drive/iGEM/Databases/Merck&Co/Validation/Per_Protein_Crossvalidation/"
print("\t",os.path.isdir(data_target_location))

print("Files: ")
data_folder = "gdrive/My Drive/iGEM/Databases/Merck&Co/Data/T1973/"

index_T1973_loc = data_folder+"paper_datasets_T1973/index_xgboost_T1973.csv"
label_T1973_loc = data_folder+"paper_datasets_T1973/y_label_xgboost_T1973.csv"

X_paper_T1973_loc = data_folder+"paper_datasets_T1973/X_paper_xgboost_T1973.csv"
X_cnn_rep_all_03_T1973_loc = data_folder+"cnn_rep_sampler_T1973/sequence_cnn_rep_all_default_03.csv"
X_cnn_rep_mut_03_T1973_loc = data_folder+"cnn_rep_sampler_T1973/sequence_cnn_rep_mut_default_03.csv"
X_ALG_Cart_tiE_T1973_loc = data_folder+"paper_datasets_T1973/X_ALG_Cart-tiE_xgboost_T1973.csv"
X_ALG_T1973_loc = data_folder+"paper_datasets_T1973/X_ALG_xgboost_T1973.csv"

print("\t",os.path.isfile(index_T1973_loc))
print("\t",os.path.isfile(label_T1973_loc))
print("\t",os.path.isfile(X_cnn_rep_all_03_T1973_loc))
print("\t",os.path.isfile(X_cnn_rep_all_03_T1973_loc))
print("\t",os.path.isfile(X_cnn_rep_mut_03_T1973_loc))
print("\t",os.path.isfile(X_ALG_Cart_tiE_T1973_loc))
print("\t",os.path.isfile(X_ALG_T1973_loc))

Check correctnes of locations: 
Folders: 
	 True
Files: 
	 True
	 True
	 True
	 True
	 True
	 True
	 True


In [0]:
#XGBoost
!pip install xgboost==0.82 #working GPU support
import xgboost as xgb

     |████████████████████████████████| 114.0MB 1.2MB/s 
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [0]:
#BayersianOptimization
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10031 sha256=f5d582c4f087471673324487a7edac99438ba6b472a38bc6a4d37837bb965b63
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


## Notebook Setup Local Runtime

In [0]:
#Imports:
import os
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import RepeatedKFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
#import seaborn as sns

import time
import datetime

In [0]:
import xgboost as xgb
from bayes_opt import BayesianOptimization

In [0]:
print("Check correctnes of locations: ")

print("Folders: ")
data_target_location = "Users/Whatever/Desktop/Google_Colab_Local/Per_Protein_Crossvalidation/"
print("\t",os.path.isdir(data_target_location))

print("Files: ")
index_T1973_loc = "Users/Whatever/Desktop/Google_Colab_Local/T1973/paper_datasets_T1973/index_xgboost_T1973.csv"
label_T1973_loc = "Users/Whatever/Desktop/Google_Colab_Local/T1973/paper_datasets_T1973/y_label_xgboost_T1973.csv"

X_paper_T1973_loc = "Users/Whatever/Desktop/Google_Colab_Local/T1973/paper_datasets_T1973/X_paper_xgboost_T1973.csv"
X_cnn_rep_all_03_T1973_loc = "Users/Whatever/Desktop/Google_Colab_Local/T1973/cnn_rep_sampler_T1973/sequence_cnn_rep_all_default_03.csv"
X_cnn_rep_mut_03_T1973_loc = "Users/Whatever/Desktop/Google_Colab_Local/T1973/cnn_rep_sampler_T1973/sequence_cnn_rep_mut_default_03.csv"
X_ALG_Cart_tiE_T1973_loc = "Users/Whatever/Desktop/Google_Colab_Local/T1973/paper_datasets_T1973/X_ALG_Cart-tiE_xgboost_T1973.csv"
X_ALG_T1973_loc = "Users/Whatever/Desktop/Google_Colab_Local/T1973/paper_datasets_T1973/X_ALG_xgboost_T1973.csv"

print("\t",os.path.isfile(index_T1973_loc))
print("\t",os.path.isfile(label_T1973_loc))
print("\t",os.path.isfile(X_cnn_rep_all_03_T1973_loc))
print("\t",os.path.isfile(X_cnn_rep_all_03_T1973_loc))
print("\t",os.path.isfile(X_cnn_rep_mut_03_T1973_loc))
print("\t",os.path.isfile(X_ALG_Cart_tiE_T1973_loc))
print("\t",os.path.isfile(X_ALG_T1973_loc))

## Prepare Data

In [0]:
#Just load data
y_label = pd.read_csv(label_T1973_loc)
y_index = pd.read_csv(index_T1973_loc)
y_index_cleaned = [x.split("@")[0].split("_")[0] for x in y_index.Mutation.values] #same wildtype protein => same name //disregard extra single mutations
X_paper_T1973 = pd.read_csv(X_paper_T1973_loc)
X_cnn_rep_all_03_T1973 = pd.read_csv(X_cnn_rep_all_03_T1973_loc).drop(["Mutation"],axis=1) #somehow mutation is in there
X_cnn_rep_mut_03_T1973 = pd.read_csv(X_cnn_rep_mut_03_T1973_loc)
X_ALG_Cart_tiE_T1973 = pd.read_csv(X_ALG_Cart_tiE_T1973_loc)
X_ALG_T1973 = pd.read_csv(X_ALG_T1973_loc)

In [0]:
unique, counts = np.unique(y_index_cleaned, return_counts=True) #to calculate the weighted MAE

In [0]:
unique

array(['1ADO', '1AKY', '1AM7', '1ANK', '1ANT', '1AQH', '1ARR', '1AVR',
       '1AYF', '1AZP', '1B5M', '1BNI', '1BPI', '1BRF', '1BTM', '1BVC',
       '1C52', '1C5G', '1C8C', '1C9O', '1CHK', '1CLW', '1CSP', '1CTS',
       '1CYO', '1E0W', '1E21', '1E65', '1ESF', '1EVQ', '1EY0', '1EZM',
       '1F6R', '1FHL', '1FLV', '1FNA', '1FTG', '1FVK', '1G5A', '1GUY',
       '1GV5', '1H0C', '1H7M', '1H8V', '1HFY', '1HFZ', '1HUE', '1HYN',
       '1I4N', '1I5T', '1IHB', '1IO2', '1IOJ', '1IRO', '1JIW', '1JNX',
       '1JU3', '1KA6', '1KDX', '1KE4', '1KF2', '1KF3', '1KF5', '1KFW',
       '1L63', '1LHM', '1LNI', '1LS4', '1LZ1', '1MJ5', '1MJC', '1N0J',
       '1OLR', '1ONC', '1OSI', '1PGA', '1PIN', '1POH', '1QLP', '1RBP',
       '1RGG', '1RN1', '1ROP', '1RRO', '1RTB', '1RTP', '1SHF', '1SHG',
       '1STN', '1SUP', '1T69', '1TCA', '1TFE', '1TPE', '1TPK', '1TTQ',
       '1URP', '1WQ5', '1YCC', '1YEA', '1YNR', '1YPI', '1YU5', '1ZDR',
       '1ZYM', '2A01', '2AFG', '2CBR', '2CI2', '2CNC', '2CPP', '2CRK',
      

In [0]:
counts

array([  5,   6,   1,   4,   1,   7,  25,   1,  18,   2,   1,  35,  15,
         2,   1,  37,   2,   2,   3,  23,   5,   5,  27,   3,  10,  12,
         2,   1,   2,   1, 129,   4,  23,   4,   4,   2,  33,   1,  53,
         2,   1,   1,  28,  16,   3,   7,   3,   2,   2,   3,   8,  14,
         1,   2,   4,   3,   3,   3,   1,   1,  43,   9,   3,   3, 184,
        15,  35,   1, 131,  99,   3,   5,   6,   4,  10,  13,  30,   9,
         9,   6,   1,  33,  15,   2,   1,   3,  49,   5,  17,   9,   5,
         3,   1,   1,  21,   2,   9,  40,  38,   1,   1,   2,   3,   4,
         2,   3,   8,   2,  18,   2,   1,   1,   1,   5,   3,   1, 106,
         1,  70,   8,   3,   2,   6,   2,  12,   4,   7,   4,  44,   3,
         4,  15,  35,  96])

In [0]:
len(set(y_index_cleaned))

134

In [0]:
X_paper_T1973.loc[np.array(y_index_cleaned)=="1AQH"]

,alpha_diff,alpha_first,alpha_second,aromatic_diff,aromatic_first,aromatic_second,beta_diff,beta_first,beta_second,branched_diff,branched_first,branched_second,bulkiness_diff,bulkiness_first,bulkiness_second,cart_cart_bonded,cart_ddg,cart_dslf_fa13,cart_fa_atr,cart_fa_dun_dev,cart_fa_dun_rot,cart_fa_dun_semi,cart_fa_elec,cart_fa_intra_atr_xover4,cart_fa_intra_elec,cart_fa_intra_rep_xover4,cart_fa_intra_sol_xover4,cart_fa_rep,cart_fa_sol,cart_hbond_bb_sc,cart_hbond_lr_bb,cart_hbond_sc,cart_hbond_sr_bb,cart_hxl_tors,cart_lk_ball,cart_lk_ball_bridge,cart_lk_ball_bridge_uncpl,cart_lk_ball_iso,cart_omega,cart_p_aa_pp,...,mono_fa_intra_rep,mono_fa_pair,mono_fa_rep,mono_fa_sol,mono_hbond_bb_sc,mono_hbond_lr_bb,mono_hbond_sc,mono_hbond_sr_bb,mono_omega,mono_p_aa_pp,mono_pro_close,mono_rama,mono_ref,mutability_diff,mutability_first,mutability_second,pi_diff,pi_first,pi_second,polarity2_diff,polarity2_first,polarity2_second,polarity_diff,polarity_first,polarity_second,r_bioluminate_delta_stability,r_bioluminate_delta_stability_coulomb,r_bioluminate_delta_stability_covalent,r_bioluminate_delta_stability_hbond,r_bioluminate_delta_stability_lipo,r_bioluminate_delta_stability_packing,r_bioluminate_delta_stability_reference,r_bioluminate_delta_stability_selfcont,r_bioluminate_delta_stability_solv_gb,r_bioluminate_delta_stability_solv_sa,r_bioluminate_delta_stability_vdw,totalarea,vdw_vol_diff,vdw_vol_first,vdw_vol_second
11,-0.29,1.35,1.06,1,0,1,0.58,0.72,1.30,0,0,0,8.10,13.57,21.67,1.654,-3.579,0.0,-10.258,0.338,-0.025,-0.527,-0.253,-0.701,0.391,-0.098,-0.099,1.615,-0.861,-0.356,0.000,-0.507,0.012,0.201,2.770,0.016,0.160,-2.029,-0.030,0.202,...,0.019,0.909,0.036,2.297,-0.923,0.358,-1.264,0.432,-0.342,-0.446,0.015,0.575,1.72,-84,102,18,2.67,3.22,5.89,-6.9,12.3,5.4,-47.80,49.90,2.10,-8.135727,-60.051019,-2.361515,0.002787,-4.486919,-1.566383e+00,-16.05,0.000000,79.637072,0,-3.259749,241.761,34.6,40.2,74.8
12,0.06,1.11,1.17,0,0,0,0.08,0.83,0.91,0,0,0,-1.43,15.71,14.28,0.583,-7.742,0.0,-4.904,0.249,-0.292,0.849,-5.965,0.091,-0.884,0.061,0.008,0.287,3.923,1.540,-0.040,-4.566,0.000,-0.010,3.788,-0.046,-0.329,-1.997,-0.082,-0.152,...,-0.035,0.119,-1.615,5.971,-2.612,-0.280,-1.162,0.565,-0.887,-0.268,0.016,0.120,-0.33,9,56,65,1.56,9.59,11.15,-0.8,11.3,10.5,2.50,49.50,52.00,-31.535345,-74.345229,8.121341,-0.417633,-19.293514,-1.792414e+00,33.96,0.000000,23.033676,0,-0.801573,389.367,15.8,61.5,77.3
13,-0.15,1.32,1.17,0,0,0,-0.19,1.10,0.91,-1,1,0,-7.12,21.40,14.28,-1.057,4.740,0.0,1.031,-0.501,2.696,0.254,0.672,0.072,0.544,-0.100,0.197,-1.333,3.407,0.729,0.009,0.224,0.000,-0.088,-1.597,0.088,0.460,0.175,-0.015,0.176,...,-0.002,0.061,0.845,3.379,-2.632,-0.145,1.635,0.511,0.301,0.387,0.044,0.179,-0.88,25,40,65,5.17,5.98,11.15,5.6,4.9,10.5,51.87,0.13,52.00,-8.191611,-80.551782,2.841743,-1.150050,-0.579331,-6.990317e-03,33.99,0.000000,41.380661,0,-4.115861,312.302,32.9,44.4,77.3
14,0.40,0.77,1.17,0,0,0,0.28,0.63,0.91,0,0,0,1.46,12.82,14.28,-0.449,1.214,0.0,-0.152,-0.065,2.478,-1.336,1.575,-0.437,0.372,0.002,-0.102,-0.406,0.135,1.507,-0.134,-0.005,-0.064,-0.009,-1.317,-0.060,-0.376,0.414,0.012,0.114,...,-0.007,0.534,1.459,-0.002,-0.065,0.748,-2.997,-0.401,0.125,0.264,0.028,0.691,-0.09,-69,134,65,5.74,5.41,11.15,-1.1,11.6,10.5,48.62,3.38,52.00,-11.556308,-5.340746,11.118272,-0.213219,-7.110965,1.465648e-04,-18.13,2.459943,10.901502,0,-5.241242,277.016,44.6,32.7,77.3
15,0.12,0.77,0.89,0,0,0,-0.08,0.63,0.55,0,0,0,-1.14,12.82,11.68,0.470,-0.726,0.0,0.287,0.053,0.003,0.299,-0.920,0.089,-0.106,-0.030,0.032,0.066,1.099,0.005,0.000,-0.019,-0.017,-0.006,0.069,-0.004,0.009,-0.208,-0.027,-0.146,...,0.002,0.104,0.891,4.926,-1.792,0.313,0.774,0.081,-1.459,0.293,0.001,-0.101,0.22,-28,134,106,-2.64,5.41,2.77,1.4,11.6,13.0,46.32,3.38,49.70,1.678307,63.631337,2.403640,-2.265510,-0.025645,7.262000e-01,-3.11,0.101785,-60.081112,0,0.297613,201.946,-2.7,32.7,30.0
16,0.18,1.17,1.35,0,0,0,-0.19,0.91,0.72,0,0,0,-0.71,14.28,13.57,-0.445,2.376,0.0,3.509,-0.050,-3.029,1.781,3.722,0.

In [0]:
X_cnn_rep_mut_03_T1973.head()

,mut_rep_0,mut_rep_1,mut_rep_2,mut_rep_3,mut_rep_4,mut_rep_5,mut_rep_6,mut_rep_7,mut_rep_8,mut_rep_9,mut_rep_10,mut_rep_11,mut_rep_12,mut_rep_13,mut_rep_14,mut_rep_15,mut_rep_16,mut_rep_17,mut_rep_18,mut_rep_19,mut_rep_20,mut_rep_21,mut_rep_22,mut_rep_23,mut_rep_24,mut_rep_25,mut_rep_26,mut_rep_27,mut_rep_28,mut_rep_29,mut_rep_30,mut_rep_31,mut_rep_32,mut_rep_33,mut_rep_34,mut_rep_35,mut_rep_36,mut_rep_37,mut_rep_38,mut_rep_39,...,mut_rep_728,mut_rep_729,mut_rep_730,mut_rep_731,mut_rep_732,mut_rep_733,mut_rep_734,mut_rep_735,mut_rep_736,mut_rep_737,mut_rep_738,mut_rep_739,mut_rep_740,mut_rep_741,mut_rep_742,mut_rep_743,mut_rep_744,mut_rep_745,mut_rep_746,mut_rep_747,mut_rep_748,mut_rep_749,mut_rep_750,mut_rep_751,mut_rep_752,mut_rep_753,mut_rep_754,mut_rep_755,mut_rep_756,mut_rep_757,mut_rep_758,mut_rep_759,mut_rep_760,mut_rep_761,mut_rep_762,mut_rep_763,mut_rep_764,mut_rep_765,mut_rep_766,mut_rep_767
0,3.554784,3.280347,3.196928,2.98941,3.001729,5.001732,2.902748,2.11822,3.150281,3.451558,3.389083,2.314466,3.4191,2.265118,3.498300,2.520946,2.348605,2.193289,5.890350,2.238925,2.443087,2.237599,3.495885,2.82216,3.063711,2.686231,3.072621,2.828349,4.113148,4.332039,2.631766,3.237533,3.435227,2.786642,3.350316,1.876877,3.797522,2.738402,3.484556,1.549978,...,2.321487,0.901711,0.182300,1.968301,1.130359,2.146721,0.549221,2.819174,0.806336,0.676917,3.168945,1.798207,1.997155,1.416786,3.225186,1.497903,1.492945,4.586280,1.775136,0.431122,1.365820,1.500617,2.299632,3.651372,1.261674,0.479557,3.075773,3.048557,1.881294,0.93438,3.359161,1.316831,0.000000,1.055476,1.873763,2.145716,2.019033,3.575753,3.583557,1.999178
1,3.554784,3.280347,3.618918,2.98941,3.001729,5.001732,2.902748,2.11822,3.150281,3.451558,3.389083,2.158927,3.4191,2.265118,3.498300,2.520946,2.348605,2.193289,5.890350,2.238925,2.443087,2.237599,3.495885,2.82216,2.961083,2.686231,3.072621,2.828349,4.113148,4.928470,2.631766,3.237533,3.435227,2.786642,3.350316,1.876877,3.797522,2.738402,3.484556,1.549978,...,2.321487,0.901711,0.381524,1.968301,1.130359,1.801425,0.549221,2.819174,0.806336,0.664614,3.556319,1.798207,1.997155,1.416786,3.225186,1.663649,1.492945,4.586280,1.775136,0.431122,1.365820,1.500617,2.361112,3.651372,1.263280,0.479557,3.353525,3.048557,1.881294,0.93438,3.357924,1.316831,0.000000,1.055476,1.873763,2.145716,1.951526,3.575753,3.789693,1.999178
2,3.554784,3.280347,3.618918,2.98941,3.001729,5.001732,2.902748,1.81807,3.150281,3.451558,3.389083,2.314466,3.4191,2.265118,3.529557,3.345332,2.348605,2.193289,5.890350,2.238925,2.443087,2.237599,3.495885,2.82216,2.961083,2.686231,3.072621,2.828349,4.113148,4.332039,2.631766,3.237533,3.435227,2.786642,3.350316,1.876877,3.797522,2.738402,3.484556,1.549978,...,2.321487,0.901711,0.245752,1.395864,1.193731,1.801425,0.207813,2.873068,1.154424,0.676917,3.220256,1.798207,1.997155,1.416786,3.225186,1.661837,1.492945,4.165377,1.775136,0.431122,1.469510,1.500617,2.298108,3.651372,1.263280,0.479557,3.353525,3.048557,1.783077,0.93438,3.359161,1.316831,0.000000,1.055476,1.873763,2.185869,2.091188,3.575753,3.789693,1.999178
3,3.554784,3.280347,3.618918,2.98941,3.001729,5.001732,2.902748,2.11822,2.561329,3.451558,3.389083,2.314466,3.4191,2.265118,3.498300,2.520946,2.348605,2.183994,5.890350,2.325331,2.645703,2.237599,3.495885,2.82216,2.961083,2.686231,3.281612,2.828349,4.113148,4.332039,2.631766,3.237533,3.435227,2.786642,3.350316,1.876877,3.797522,2.738402,3.353157,1.549978,...,2.583310,0.980569,0.245752,1.968301,1.130359,1.801425,0.549221,2.819174,0.806336,0.676917,3.220256,1.673476,1.985375,1.412786,3.361702,1.661837,1.291652,4.586280,1.621299,0.431122,1.342825,1.492803,2.298108,3.651372,1.263280,0.545775,3.353525,2.893650,1.881294,0.91506,3.359161,1.316831,0.068523,0.952350,1.883678,2.145716,1.976812,3.575753,3.789693,2.039857
4,3.554784,3.280347,3.618918,2.98941,3.001729,5.001732,2.902748,2.11822,3.150281,3.451558,3.270357,2.314466,3.4191,2.265118,3.498300,2.520946,2.348605,2.193289,4.783499,2.238925,2.443087,2.237599,3.495885,

## Supplementary functions

In [0]:
def nested_k_fold_per_proteincross_validation_raw(data_x, data_y, y_index, protein_label, eval_metric="mae", scale_label=False, inner_fold=3, bayes_init_points=15, bayes_iterations=10, bayes_acquisition_function='ei', k_fold_seed=42375):  
  print("Evaluation metric: ", eval_metric)
  print("Inner folds: ", inner_fold)
  print("Bayesian init points: ", bayes_init_points)
  print("Bayesian iterations: ", bayes_iterations)
  print("Bayes acquisition function: ", bayes_acquisition_function)

  # Variable for xgb_evalutate, unfortunately cannot be a direct argument
  bayesian_training_data = None
  # Supplementary function for hyperparameter tuning. Corresponds in essense to inner cross validation.
  def xgb_evaluate(max_depth, gamma, colsample_bytree, subsample, eta):
      params = {'eval_metric': eval_metric,
                'objective':'reg:linear',
                'tree_method': 'gpu_hist',
                'max_depth': int(max_depth),
                'eta': eta,
                'gamma': gamma,
                'colsample_bytree': colsample_bytree,
                'subsample': subsample}
      # This corresponds to the inner cross validation cycle for hyperparameter tuning:
      cv_result = xgb.cv(params, bayesian_training_data, num_boost_round=100, nfold=inner_fold)    
      # Bayesian optimization only knows how to maximize, not minimize, thus negate score:
      return -1.0 * cv_result['test-mae-mean'].iloc[-1]
    
  # Bounds for the bayesian optimization
  bay_bounds={'eta': (0.01, 0.3),
              'max_depth': (5, 10), 
              'gamma': (0.0, 1.0),
              'colsample_bytree': (0.5, 1.0),
              'subsample': (0.7, 1.0)}
  
  #saves result for each outer cross val
  protein_label = np.array(protein_label) 
  proteins = np.sort(np.unique(protein_label)) #this will determine the number of folds, since this is protein based cross-val
  print("Totoal Folds: ",len(proteins))

  scores_mae = [] 
  scores_rmse = []
  best_parameters = []
  
  indeces_raw = []
  labels_raw = []
  preds_raw = []
  prots_raw = []
  
  total_fold_counter = 0

  # Main loop for trials and outer outer cross_val
  for curr_protein in proteins:
    #Measure runtime
    start_time = time.time()
    
    # Print current status
    total_fold_counter += 1
    print("Fold number: {}, Protein: {}".format(total_fold_counter,curr_protein))
    
    # Calculate current indices
    train_index = protein_label != curr_protein
    test_index = protein_label == curr_protein
    
    # Select current train and test data
    X_train, X_test = data_x.loc[train_index], data_x.loc[test_index]
    y_train, y_test = data_y.loc[train_index], data_y.loc[test_index]
    
    # Document all results
    indeces_raw.extend(y_index[test_index].values)
    labels_raw.extend(y_test.values)
    prots_raw.extend([curr_protein]*len(y_test))
    
    # Scale target label, if requested
    standard_scaler = preprocessing.StandardScaler()
    if scale_label:
      y_train = standard_scaler.fit_transform(y_train.reshape(-1, 1)).reshape((len(y_train))) #only fit on y_train to prevent information leakage
    
    # Prepare data for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)
    
    # Bayesian optimize in 5 fold cross validation
    xgb_bo = BayesianOptimization(f=xgb_evaluate, 
                                  pbounds=bay_bounds,
                                  random_state=((total_fold_counter*k_fold_seed)%1000))
    bayesian_training_data = dtrain
    xgb_bo.maximize(init_points=bayes_init_points, n_iter=bayes_iterations, acq=bayes_acquisition_function) #default: 20,20,'ei'
    
    # Extract optimal parameters
    params_opt = xgb_bo.max['params']
    best_parameters.append(params_opt) #save best parameters
    params_opt['max_depth'] = int(params_opt['max_depth'])
    params_opt['eval_metric'] = eval_metric
    params_opt['objective'] = 'reg:linear'
    params_opt['tree_method'] = 'gpu_hist'
    
    # Train a new model with the best parameters from the search
    model = xgb.train(params_opt, dtrain, num_boost_round=1000)

    # Predict on testing set
    y_pred = model.predict(dtest)
    if scale_label:
      y_pred = standard_scaler.inverse_transform(y_pred.reshape(-1, 1))
      
    # Document all results
    preds_raw.extend(y_pred)

    # Report testing and training scores
    scores_mae.append(mean_absolute_error(y_test, y_pred))
    scores_rmse.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    print("MAE: ", scores_mae[-1], " RMSE: ",scores_rmse[-1], " Runtime: ", str(datetime.timedelta(seconds=(time.time()-start_time))))
  
  print()
  results = pd.DataFrame(best_parameters)
  results["scores_mae"] = scores_mae
  results["scores_rmse"] = scores_rmse
  results["protein"] = proteins
  
  results_doc = pd.DataFrame([indeces_raw,prots_raw,labels_raw,preds_raw])
  results_doc = results_doc.T
  results_doc.columns = ["indices","protein","labels","preds"]
  return results, results_doc

## Evaluate different models (based on source data)

In [0]:
# Test indices
y_index_cleaned = np.array(y_index_cleaned)
selection = (y_index_cleaned=="1AKY") | (y_index_cleaned=="1AYF")

### Model cnn rep: (A3)

In [0]:
model_cnn_rep_all_results, raw_results = nested_k_fold_per_proteincross_validation_raw(X_cnn_rep_all_03_T1973, y_label, y_index, pd.Series(y_index_cleaned))

Evaluation metric:  mae
Inner folds:  3
Bayesian init points:  15
Bayesian iterations:  10
Bayes acquisition function:  ei
Totoal Folds:  134
Fold number: 1, Protein: 1ADO
|   iter    |  target   | colsam... |    eta    |   gamma   | max_depth | subsample |
-------------------------------------------------------------------------------------
|  1        | -3.258    |  0.6016   |  0.03383  |  0.0537   |  7.328    |  0.7635   |
|  2        | -3.348    |  0.7643   |  0.2832   |  0.2935   |  5.067    |  0.8559   |
|  3        | -3.4      |  0.9518   |  0.2416   |  0.7593   |  5.313    |  0.7975   |
|  4        | -3.555    |  0.5583   |  0.01254  |  0.4712   |  5.634    |  0.8909   |
|  5        | -3.379    |  0.697    |  0.2568   |  0.6236   |  9.847    |  0.8862   |
|  6        | -3.212    |  0.8253   |  0.07749  |  0.3247   |  8.73     |  0.8518   |
|  7        | -3.345    |  0.636    |  0.1644   |  0.882    |  8.065    |  0.8558   |
|  8        | -3.23     |  0.8363   |  0.1261   |  0.9

In [0]:
model_cnn_rep_all_results.describe()

,colsample_bytree,eta,gamma,max_depth,subsample,scores_mae,scores_rmse
count,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000
mean,0.720813,0.084367,0.495958,8.156716,0.858184,4.713231,5.258389
std,0.132219,0.030564,0.276891,0.916470,0.069776,2.906374,3.023606
min,0.513764,0.030529,0.020533,5.000000,0.700936,0.100895,0.100895
25%,0.605871,0.063112,0.324713,8.000000,0.809113,2.342826,2.814712
50%,0.729514,0.077492,0.414122,8.000000,0.851849,4.275064,5.101585
75%,0.822126,0.092901,0.782172,9.000000,0.906830,6.107675,7.079963
max,0.990882,0.162371,0.988374,9.000000,1.000000,13.505624,13.666389


In [0]:
raw_results.tail()

,indices,protein,labels,preds
1968,[GKR3BB@A@V182I],GKR3BB,[-8.12685],-1.648
1969,[GKR3BB@A@Y164N],GKR3BB,[-12.130958],-2.82433
1970,[GKR3BB@A@Y27N],GKR3BB,[-12.0979425],-4.63254
1971,[GKR3BB@A@Y27W],GKR3BB,[-10.9563065],-1.5267
1972,[GKR3BB@A@Y87P],GKR3BB,[-15.518509],-4.5196


In [0]:
raw_results.describe()

,indices,protein,labels,preds
count,1973,1973,1973,1973.00000
unique,1973,134,397,1973.00000
top,[1YCC@A@F10Y],1L63,[0.0],-1.57299
freq,1,184,65,1.00000


In [0]:
raw_results.to_csv(data_target_location+"raw_results_cnn_rep_all_xgboost_T1973_ppcv.csv")

In [0]:
raw_results = pd.read_csv(data_target_location+"raw_results_cnn_rep_all_xgboost_T1973_ppcv.csv")
raw_results.head()

,Unnamed: 0,indices,protein,labels,preds
0,0,['1ADO@A@D128A'],1ADO,[-11.9],-1.727950
1,1,['1ADO@A@D128G'],1ADO,[-12.7],-1.494252
2,2,['1ADO@A@D128N'],1ADO,[-11.7],-1.779992
3,3,['1ADO@A@D128Q'],1ADO,[-11.6],-1.247529
4,4,['1ADO@A@D128V'],1ADO,[-12.2],-1.984397


In [0]:
raw_results["labels"] = list(map(lambda x: float(x[1:-1]), raw_results["labels"]))
raw_results["indices"] = list(map(lambda x: str(x[1:-1]), raw_results["indices"]))
raw_results = raw_results[raw_results.columns[1:]]
raw_results.head()

,indices,protein,labels,preds
0,'1ADO@A@D128A',1ADO,-11.9,-1.727950
1,'1ADO@A@D128G',1ADO,-12.7,-1.494252
2,'1ADO@A@D128N',1ADO,-11.7,-1.779992
3,'1ADO@A@D128Q',1ADO,-11.6,-1.247529
4,'1ADO@A@D128V',1ADO,-12.2,-1.984397


In [0]:
raw_results.to_csv(data_target_location+"raw_results_cnn_rep_all_xgboost_T1973_ppcv_NEW.csv", index=False)

**Auc calculation:**

In [0]:
tmp_ground_t = raw_results["labels"].values > 1  #as done in McGuinness et al. 2018

In [0]:
fpr, tpr, thresholds = metrics.roc_curve(tmp_ground_t, raw_results["preds"].values)
metrics.auc(fpr, tpr)

0.564146475983714

In [0]:
tmp_ground_t = raw_results["labels"].values > 0

In [0]:
fpr, tpr, thresholds = metrics.roc_curve(tmp_ground_t, raw_results["preds"].values)
metrics.auc(fpr, tpr)

0.5913671904846802

**Pearson Correlation Coefficient calculation:**

In [0]:
raw_results = pd.read_csv(data_target_location+"raw_results_cnn_rep_all_xgboost_T1973_ppcv_NEW.csv")

In [0]:
tmp_ground_t = raw_results["labels"].values > 0

In [0]:
pearsonr(tmp_ground_t, raw_results["preds"].values) #first value

(0.15092142632104474, 1.6049529411234368e-11)

### Model paper ppcv: (S_E)

In [0]:
model_paper_results, raw_results = nested_k_fold_per_proteincross_validation_raw(X_paper_T1973, y_label, y_index, pd.Series(y_index_cleaned))

Evaluation metric:  mae
Inner folds:  3
Bayesian init points:  15
Bayesian iterations:  10
Bayes acquisition function:  ei
Totoal Folds:  134
Fold number: 1, Protein: 1ADO
|   iter    |  target   | colsam... |    eta    |   gamma   | max_depth | subsample |
-------------------------------------------------------------------------------------
|  1        | -2.984    |  0.6016   |  0.03383  |  0.0537   |  7.328    |  0.7635   |
|  2        | -3.145    |  0.7643   |  0.2832   |  0.2935   |  5.067    |  0.8559   |
|  3        | -3.124    |  0.9518   |  0.2416   |  0.7593   |  5.313    |  0.7975   |
|  4        | -3.368    |  0.5583   |  0.01254  |  0.4712   |  5.634    |  0.8909   |
|  5        | -3.151    |  0.697    |  0.2568   |  0.6236   |  9.847    |  0.8862   |
|  6        | -2.925    |  0.8253   |  0.07749  |  0.3247   |  8.73     |  0.8518   |
|  7        | -3.061    |  0.636    |  0.1644   |  0.882    |  8.065    |  0.8558   |
|  8        | -2.969    |  0.8363   |  0.1261   |  0.9

In [0]:
model_paper_results.describe()

,colsample_bytree,eta,gamma,max_depth,subsample,scores_mae,scores_rmse
count,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000
mean,0.691819,0.073771,0.391222,7.141791,0.838945,4.036521,4.652058
std,0.136105,0.029340,0.252522,1.344177,0.074684,2.600242,2.786744
min,0.511630,0.030558,0.007143,5.000000,0.700000,0.019424,0.019424
25%,0.596137,0.056826,0.236535,6.000000,0.799472,2.266639,2.768041
50%,0.624958,0.070909,0.324713,7.000000,0.824399,3.523560,4.084186
75%,0.825265,0.083646,0.545862,8.000000,0.892402,5.235333,6.224659
max,1.000000,0.224561,0.975458,9.000000,0.996978,15.651287,15.853417


In [0]:
raw_results.tail()

,indices,protein,labels,preds
1968,[GKR3BB@A@V182I],GKR3BB,[-8.12685],0.852664
1969,[GKR3BB@A@Y164N],GKR3BB,[-12.130958],-8.88694
1970,[GKR3BB@A@Y27N],GKR3BB,[-12.0979425],-4.55905
1971,[GKR3BB@A@Y27W],GKR3BB,[-10.9563065],1.44531
1972,[GKR3BB@A@Y87P],GKR3BB,[-15.518509],-8.51518


In [0]:
raw_results.describe()

,indices,protein,labels,preds
count,1973,1973,1973,1973.000000
unique,1973,134,397,1973.000000
top,[1YCC@A@F10Y],1L63,[0.0],-1.557626
freq,1,184,65,1.000000


In [0]:
raw_results.to_csv(data_target_location+"raw_results_paper_xgboost_T1973_ppcv.csv")

In [0]:
raw_results = pd.read_csv(data_target_location+"raw_results_paper_xgboost_T1973_ppcv.csv")
raw_results.head()

,Unnamed: 0,indices,protein,labels,preds
0,0,['1ADO@A@D128A'],1ADO,[-11.9],-2.259035
1,1,['1ADO@A@D128G'],1ADO,[-12.7],-0.702524
2,2,['1ADO@A@D128N'],1ADO,[-11.7],0.054538
3,3,['1ADO@A@D128Q'],1ADO,[-11.6],3.096903
4,4,['1ADO@A@D128V'],1ADO,[-12.2],-0.363375


In [0]:
raw_results["labels"] = list(map(lambda x: float(x[1:-1]), raw_results["labels"]))
raw_results["indices"] = list(map(lambda x: str(x[1:-1]), raw_results["indices"]))
raw_results = raw_results[raw_results.columns[1:]]
raw_results.head()

,indices,protein,labels,preds
0,'1ADO@A@D128A',1ADO,-11.9,-2.259035
1,'1ADO@A@D128G',1ADO,-12.7,-0.702524
2,'1ADO@A@D128N',1ADO,-11.7,0.054538
3,'1ADO@A@D128Q',1ADO,-11.6,3.096903
4,'1ADO@A@D128V',1ADO,-12.2,-0.363375


In [0]:
raw_results.to_csv(data_target_location+"raw_results_paper_xgboost_T1973_ppcv_NEW.csv", index=False)

**Auc calculation:**

In [0]:
tmp_ground_t = raw_results["labels"] > 1 #as done in McGuinness et al. 2018

In [0]:
fpr, tpr, thresholds = metrics.roc_curve(tmp_ground_t, raw_results["preds"].values)
metrics.auc(fpr, tpr)

0.7215688134307123

In [0]:
tmp_ground_t = raw_results["labels"] > 0

In [0]:
fpr, tpr, thresholds = metrics.roc_curve(tmp_ground_t, raw_results["preds"].values)
metrics.auc(fpr, tpr)

0.7387667778824848

**Pearson Correlation Coefficient calculation:**

In [0]:
raw_results = pd.read_csv(data_target_location+"raw_results_paper_xgboost_T1973_ppcv_NEW.csv")

In [0]:
tmp_ground_t = raw_results["labels"].values > 0

In [0]:
pearsonr(tmp_ground_t, raw_results["preds"].values) #first value

(0.3547833326557596, 1.3354944200070742e-59)